## Load Guinea-Bissau data and save as numpy file

In [2]:
import numpy as np
import pandas as pd
from pandas import merge 
from os import listdir
from numpy import genfromtxt, random

In [3]:
datadir = "/media/windows-share/EEG/EEGs_Nigeria_cleaned"
outputdir = "/media/windows-share/EEG/EEGs_Nigeria_np"
filenames = listdir(datadir)
D = []
sf = 128
nc = 14
#Nfiles = len(filenames)
#X = np.zeros((Nfiles,maxtslength,nc)) 

In [4]:
id = list(map(int,list(map(lambda file: file[file.find('id')+2:file.find('dur')-1],filenames))))

In [5]:
dur = list(map(int,list(map(lambda file: file[file.find('dur')+3:file.find('epoch')-1],filenames))))
#epoch = list(map(int,list(map(lambda file: file[file.find('epoch')+5:file.find('gro')-1],filenames))))

In [6]:
group = list(map(str,list(map(lambda file: file[file.find('gro')+3:file.find('.csv')],filenames))))

In [7]:
protocol = list(map(str,list(map(lambda file: file[file.find('yes')+3:file.find('id')-1],filenames))))

In [8]:
mydata = id, dur, group
df = pd.DataFrame.from_items([('id',id),('dur',dur),('group',group),('filenames',filenames),
                              ('protocol',protocol)])

In [15]:
logstructure = []
for mindur in [10]: # minimum duration of an epoch in seconds
    for protocol in ['open','closed']:
        df2 = df[(df['protocol']==protocol) & (df['dur'] >= mindur)]
        maxtslength = mindur * sf
        #Identify training, test and validation group
        con = np.unique(df2[df2['group'] == 'control']['id'])
        epi = np.unique(df2[df2['group'] == 'epilepsy']['id'])
        Nid = len(con) + len(epi) #number of ids
        prop = 0.5 # len(con) / Nid #proportion of controls
        random.seed(300)
        def getids(x,y,prop,N):
            ix = np.sort(np.random.choice(x,round(N*prop),replace=False))
            iy = np.sort(np.random.choice(y,round(N*(1-prop)),replace=False))
            if (len(ix)+len(iy)) < 20:
                print(prop,N,len(x),len(y))
            x = [x for i,x in enumerate(x) if x not in ix]    
            y = [x for i,x in enumerate(y) if x not in iy]    
            icon = np.concatenate((ix,iy))
            return icon, x, y
        ival, con, epi = getids(con,epi,prop,N=20) # validation set
        ites, con, epi = getids(con,epi,prop,N=20) # test set
        itra = np.concatenate((con, epi)) # training set
        #print(len(ival),len(ites),len(itra))
        # Now use identifies per group to load the data
        for subset in ['train','valid','test']:
            conditionname = subset+'_'+str(mindur)+'seconds_'+protocol
            if subset == 'train':
                tmp = df2[df2.id.isin(itra)]
                filenames = tmp['filenames']
            if subset == 'valid':
                tmp = df2[(df2.id.isin(ival))]
                tmp = tmp.sort_values(by=['id']).groupby('id').first() # select first available epoch
                filenames = tmp['filenames']
            if subset == 'test':
                tmp = df2[(df2.id.isin(ites))]
                tmp = tmp.sort_values(by=['id']).groupby('id').first() # select first available epoch
                filenames = tmp['filenames']
            X = np.zeros((0,maxtslength,nc)) #len(filenames)
            y = np.zeros((0,1)) #len(filenames)
            
            print(conditionname + ' ' + str(len(filenames)))
            for file in filenames:
                path = datadir + '/' + file
                D = pd.read_csv(path, sep=',',header=0,usecols=list(range(0,14)))
                if D.shape[0] > maxtslength:
                    slicesize = sf * 10
                    for slicei in range(int(len(D)/slicesize)):
                        sta = (((slicei)*slicesize))
                        end = ((slicei+1)*slicesize)
                        D2 = np.array(D[sta:end]) # take first part or should these be a random selection?
                    #D = np.array(D[0:maxtslength]) # take first part or should these be a random selection?
                        D2 = np.reshape(D2,(1,D2.shape[0],D2.shape[1]))
                        m = D2.mean(axis=1,keepdims=True)
                        D2 = D2 - m # subtract mean
                        X = np.vstack((X,D2))
                        logstructure.append([subset,mindur,protocol,file])
                        diagnosis = tmp.group[(tmp.filenames == file)]
                        y = np.vstack((y,diagnosis))
            
            fnameX = outputdir + '/X_' + conditionname
            fnamey = outputdir + '/y_' + conditionname
            np.save(file=fnameX,arr=X)
            #y = np.array(tmp['group'])
            np.save(file=fnamey,arr=y)            
np.savetxt(outputdir + '/log.csv', logstructure,
           delimiter=",", fmt='%s')

train_10seconds_open 272
valid_10seconds_open 20
test_10seconds_open 20
train_10seconds_closed 255
valid_10seconds_closed 20
test_10seconds_closed 20


In [11]:
print(fnameX)
print(fnamey)
testreadX = np.load(file=fnameX+'.npy')
testready = np.load(file=fnamey+'.npy')

/media/windows-share/EEG/EEGs_Nigeria_np/X_test_10seconds_closed
/media/windows-share/EEG/EEGs_Nigeria_np/y_test_10seconds_closed


In [12]:
print(testreadX.shape)
print(testready.shape)

(163, 1280, 14)
(163, 1)
